In [1]:
import torch.utils.data as td
import numpy as np

# DistributedSampler
As with all samplers, it only makes sense to use this sampler with map-style datasets. It is supposed to be used in a data-parallel setting where there are multiple processes with a copy of the DNN processing a subset of the overall data. When the `DistributedSampler` is initialized in one of the processes which is identified by its rank, the sampler does the following:

  1. Shuffles the indexes in the dataset (by default `shuffle=True` in the ctor).
  2. Gets the `rank` and the `world_size` either from the environment variables or from the passed in arguments.
  3. Chops up the shuffled list of indexes into `world_size` chunks and allocates each chunk to a `rank`.

Now, when the data loader asks it to give an index so it can get that instance from the dataset, the sampler will dole out the indexes allocated to its own `rank`. As an aside, this sampler is not a batched sampler, so the data loader will still have to use some way of batching individual instances, e.g., using the default `BatchSampler`.

A sampler is initialized in each process of every rank. The only way the same chunks are allocated to the same `rank` is if the indexes are shuffled using the same random seed. For this reason, the ctor takes a default seed of $0$, but of course I can give it any seed I want to.

In [2]:
class Simple(td.Dataset):
    def __init__(self, m=100, n=3):
        self._x = np.array([np.full(n, i + 1) for i in range(m)])
        self._y = np.random.choice([0, 1], size=m, p=[0.7, 0.3])

    def __len__(self):
        return self._x.shape[0]

    def __getitem__(self, idx):
        return self._x[idx], self._y[idx]


In [3]:
ds = Simple(m=10)
for x, y in ds:
    print(x, y)

[1 1 1] 1
[2 2 2] 0
[3 3 3] 1
[4 4 4] 1
[5 5 5] 1
[6 6 6] 0
[7 7 7] 0
[8 8 8] 0
[9 9 9] 0
[10 10 10] 0


To demonstrate the related concepts in this notebook I'll pass in the world size and the rank of each sampler in its ctor. Typically these don't need to be specified because they are picked up from the environment variables. In the call to the `DistributedSampler` ctor in the cells below, `shuffle=True` and `seed=0` is passed in by default. Because of this seed, everytime I run these two cells, rank 0 and rank 1 will have the same "shuffled" data.

##### Rank 0 data:

```
[tensor([[20, 20, 20],
        [ 7,  7,  7],
        [ 6,  6,  6],
        [12, 12, 12]]), tensor([1, 0, 0, 0])]
[tensor([[15, 15, 15],
        [22, 22, 22],
        [16, 16, 16],
        [23, 23, 23]]), tensor([0, 1, 0, 1])]
[tensor([[14, 14, 14],
        [ 8,  8,  8],
        [ 4,  4,  4],
        [21, 21, 21]]), tensor([1, 1, 0, 0])]
[tensor([[1, 1, 1]]), tensor([0])]
```

##### Rank 1 data:

```
[tensor([[17, 17, 17],
        [18, 18, 18],
        [25, 25, 25],
        [ 9,  9,  9]]), tensor([1, 0, 0, 1])]
[tensor([[13, 13, 13],
        [ 3,  3,  3],
        [ 2,  2,  2],
        [11, 11, 11]]), tensor([0, 0, 1, 0])]
[tensor([[10, 10, 10],
        [19, 19, 19],
        [ 5,  5,  5],
        [24, 24, 24]]), tensor([1, 0, 1, 0])]
[tensor([[20, 20, 20]]), tensor([0])]
```

In [11]:
ds0 = Simple(m=25)
sampler0 = td.distributed.DistributedSampler(ds0, num_replicas=2, rank=0)
dl0 = td.DataLoader(ds0, batch_size=4, sampler=sampler0)
for batch in dl0:
    print(batch)

[tensor([[20, 20, 20],
        [ 7,  7,  7],
        [ 6,  6,  6],
        [12, 12, 12]]), tensor([1, 0, 0, 0])]
[tensor([[15, 15, 15],
        [22, 22, 22],
        [16, 16, 16],
        [23, 23, 23]]), tensor([0, 1, 0, 1])]
[tensor([[14, 14, 14],
        [ 8,  8,  8],
        [ 4,  4,  4],
        [21, 21, 21]]), tensor([1, 1, 0, 0])]
[tensor([[1, 1, 1]]), tensor([0])]


In [12]:
ds1 = Simple(m=25)
sampler1 = td.distributed.DistributedSampler(ds1, num_replicas=2, rank=1)
dl1 = td.DataLoader(ds1, batch_size=4, sampler=sampler1)
for batch in dl1:
    print(batch)

[tensor([[17, 17, 17],
        [18, 18, 18],
        [25, 25, 25],
        [ 9,  9,  9]]), tensor([1, 0, 0, 1])]
[tensor([[13, 13, 13],
        [ 3,  3,  3],
        [ 2,  2,  2],
        [11, 11, 11]]), tensor([0, 0, 1, 0])]
[tensor([[10, 10, 10],
        [19, 19, 19],
        [ 5,  5,  5],
        [24, 24, 24]]), tensor([1, 0, 1, 0])]
[tensor([[20, 20, 20]]), tensor([0])]


To see how the indexes are partitioned across the different processes, try this out without shuffling. I'll see that the sampler assignes the indexes in a round robin fashion to each rank. Of course if `shuffle=False` the seed does not matter anymore.

In [15]:
ds0 = Simple(m=25)
sampler0 = td.distributed.DistributedSampler(ds0, num_replicas=3, rank=0, shuffle=False)
dl0 = td.DataLoader(ds0, batch_size=4, sampler=sampler0)
for batch in dl0:
    print(batch)

[tensor([[ 1,  1,  1],
        [ 4,  4,  4],
        [ 7,  7,  7],
        [10, 10, 10]]), tensor([0, 0, 0, 0])]
[tensor([[13, 13, 13],
        [16, 16, 16],
        [19, 19, 19],
        [22, 22, 22]]), tensor([1, 0, 1, 1])]
[tensor([[25, 25, 25]]), tensor([0])]


In [16]:
ds1 = Simple(m=25)
sampler1 = td.distributed.DistributedSampler(ds1, num_replicas=3, rank=1, shuffle=False)
dl1 = td.DataLoader(ds1, batch_size=4, sampler=sampler1)
for batch in dl1:
    print(batch)

[tensor([[ 2,  2,  2],
        [ 5,  5,  5],
        [ 8,  8,  8],
        [11, 11, 11]]), tensor([0, 1, 0, 0])]
[tensor([[14, 14, 14],
        [17, 17, 17],
        [20, 20, 20],
        [23, 23, 23]]), tensor([0, 0, 1, 1])]
[tensor([[1, 1, 1]]), tensor([0])]


In [17]:
ds2 = Simple(m=25)
sampler2 = td.distributed.DistributedSampler(ds2, num_replicas=3, rank=2, shuffle=False)
dl2 = td.DataLoader(ds2, batch_size=4, sampler=sampler2)
for batch in dl2:
    print(batch)

[tensor([[ 3,  3,  3],
        [ 6,  6,  6],
        [ 9,  9,  9],
        [12, 12, 12]]), tensor([0, 0, 1, 1])]
[tensor([[15, 15, 15],
        [18, 18, 18],
        [21, 21, 21],
        [24, 24, 24]]), tensor([0, 0, 0, 1])]
[tensor([[2, 2, 2]]), tensor([0])]
